In [182]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [183]:
warehouse_input = widgets.IntText(
    value=1,
    description='Warehouse Number:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='175px')
)

In [184]:
#Local Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_profit():
    print("Showing profit...")

    url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"
    userVal = {"user_id" : 10}
    response = requests.post(url+"warehouse/1/profit", json = userVal)
    warehouses = response.json()

    warehouses = {
        "profit": [-15226, 0, 0, -6160, 0, 0, 0, 0, 0, 0, 0, 713],
        "profit_year": [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 3001]
    }

    df = pd.DataFrame(warehouses)

    plt.figure(figsize=(10, 5))
    plt.bar(df['profit_year'].astype(str), df['profit'], color='skyblue')
    plt.xlabel('Year')
    plt.ylabel('Profit')
    plt.title('Profit by Year')
    plt.xticks(rotation=45)
    plt.tight_layout()  # Adjusts plot to ensure everything fits without overlapping
    plt.show()

def show_top_expensive_racks():
    warehouse_number = warehouse_input.value
    print("Showing top 5 expensive racks...")
    userVal = {"user_id" : 10}
    response = requests.post(url+ f"warehouse/{warehouse_number}/rack/expensive", json = userVal)
    racks_data = response.json()
    df = pd.DataFrame(racks_data)
    display(df)

def show_bottom_parts_type():
    warehouse_number = warehouse_input.value
    print("Showing bottom 3 parts type/material...")
    userVal = {"user_id" : 10}
    response = requests.post(url+f"warehouse/{warehouse_number}/rack/material", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_racks_under_capacity():
    warehouse_number = warehouse_input.value
    print("Showing top 5 racks under 25% capacity...")
    userVal = {"user_id" : 10}
    response = requests.post(url+f"warehouse/{warehouse_number}/rack/lowstock", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_suppliers_warehouse():
    warehouse_number = warehouse_input.value
    print("Showing top 3 suppliers that supplied to the warehouse...")
    userVal = {"user_id" : 10}
    response = requests.post(url+f"warehouse/{warehouse_number}/transaction/suppliers", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_users_exchanges():
    warehouse_number = warehouse_input.value
    print("Showing top 3 users that receives the most exchanges...")
    userVal = {"user_id" : 10}
    response = requests.post(url+f"warehouse/{warehouse_number}/users/receivesmost", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

def show_top_days_smallest_incoming():
    warehouse_number = warehouse_input.value
    print("Showing top 3 days with the smallest incoming transactions' cost...")
    userVal = {"user_id" : 10}
    response = requests.post(url+f"warehouse/{warehouse_number}/transaction/leastcost", json = userVal)
    warehouses = response.json()
    df = pd.DataFrame(warehouses)
    display(df)

In [185]:
# Global Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_top_warehouses_racks():
    print("Showing top 10 warehouses with the most racks...")
    gstat = "most/rack"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_incoming_transactions():
    print("Showing top 5 warehouse that has the most incoming transactions...")
    gstat = "most/incoming"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_delivers_exchanges():
    print("Showing top 5 warehouse that delivers the most exchanges...")
    gstat = "most/deliver"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
        
def show_top_users_transactions():
    print("Showing top 3 users that made the most transactions...")
    gstat = "most/transactions"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

def show_top_warehouses_least_outgoing_transactions():
    print("Showing top 3 warehouses with the least outgoing transactions...")
    gstat = "least/outgoing"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
def show_top_warehouses_cities():
    print("Showing top 3 warehouses cities with the most transactions...")
    gstat = "most/city"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)

In [186]:
#Transactions Data Tables
  
def show_latest_to_oldest():
    print("Showing latest to oldest transactions...")

def show_oldest_to_latest():
    print("Showing oldest to latest transactions...")

In [187]:
#Prices Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_part_prices():
    print("Showing part prices...")
    

def show_parts_supplied_by_suppliers():
    print("Showing parts supplied by suppliers...")

In [188]:
# Dropdown Menus

local_stats_dropdown = widgets.Dropdown(
    options=['Profit', 'Top 5 Expensive Racks', 'Bottom 3 Parts Type/Material', 'Top 5 Racks Under 25% Capacity', 'Top 3 Suppliers Per Warehouse', 'Top 3 Users With Exchanges', 'Top 3 Days Smallest Incoming'],
    description='Local Stats:',
)

global_stats_dropdown = widgets.Dropdown(
    options=['Top 10 Warehouses With Most Racks', 'Top 5 Warehouses With Most Incoming Transactions', 'Top 5 Warehouses That Delivers The Most Exchanges','Top 3 Users That Made The Most Transactions','Top 3 Warehouses With The Least Outgoing Transactions', 'Top 3 Warehouses Cities With The Most Transactions'],
    description='Global Stats:',
)

transactions_dropdown = widgets.Dropdown(
    options=['Latest to Oldest', 'Oldest to Latest'],
    description='Transactions:',
)

parts_dropdown = widgets.Dropdown(
    options=['Part Prices', 'Parts Supplied by Suppliers'],
    description='Parts:',
)

In [189]:
# Create an output widget to display results

output = widgets.Output()

def on_local_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Profit':
                show_profit()
            elif change['new'] == 'Top 5 Expensive Racks':
                show_top_expensive_racks()
            elif change['new'] == 'Bottom 3 Parts Type/Material':
                show_bottom_parts_type()
            elif change['new'] == 'Top 5 Racks Under 25% Capacity':
                show_racks_under_capacity()
            elif change['new'] == 'Top 3 Suppliers Per Warehouse':
                show_top_suppliers_warehouse()
            elif change['new'] == 'Top 3 Users With Exchanges':
                show_top_users_exchanges()
            elif change['new'] == 'Top 3 Days Smallest Incoming':
                show_top_days_smallest_incoming()


def on_global_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Top 10 Warehouses With Most Racks':
                show_top_warehouses_racks()
            elif change['new'] == 'Top 5 Warehouses With Most Incoming Transactions':
                show_top_warehouses_incoming_transactions()
            elif change['new'] == 'Top 5 Warehouses That Delivers The Most Exchanges':
                show_top_warehouses_delivers_exchanges()
            elif change['new'] == 'Top 3 Users That Made The Most Transactions':
                show_top_users_transactions()
            elif change['new'] == 'Top 3 Warehouses With The Least Outgoing Transactions':
                show_top_warehouses_least_outgoing_transactions()
            elif change['new'] == 'Top 3 Warehouses Cities With The Most Transactions':
                show_top_warehouses_cities()

def on_transactions_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Latest to Oldest':
                show_latest_to_oldest()
            elif change['new'] == 'Oldest to Latest':
                show_oldest_to_latest()

def on_parts_dropdown_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Part Prices':
                show_part_prices()
            elif change['new'] == 'Parts Supplied by Suppliers':
                show_parts_supplied_by_suppliers()

In [190]:
# Register the event handlers

local_stats_dropdown.observe(on_local_stats_change, names='value')
global_stats_dropdown.observe(on_global_stats_change, names='value')
transactions_dropdown.observe(on_transactions_change, names='value')
parts_dropdown.observe(on_parts_dropdown_change, names='value')

# Layout the widgets

title_widget = widgets.HTML(
    value="<h2 style='text-align: center;'>Datavengers - Phase 3</h2>"
)
dropdowns_vbox = widgets.HBox([local_stats_dropdown, global_stats_dropdown, transactions_dropdown, parts_dropdown],layout=widgets.Layout(justify_content='center', width='100%'))
main_vbox = widgets.VBox([title_widget, warehouse_input, dropdowns_vbox, output])

# Display the main layout

display(main_vbox)